In [ ]:
using QuantumCollocation
using NamedTrajectories
using CairoMakie

In [ ]:
ops = Options()
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 0.001
ops.print_level = 4
ops.hessian_approximation = "limited-memory"

integrator_kwargs = Dict(
    :order => 4,
    :autodiff => false,
    :calculate_pade_operators_structure => true,
    :G_function => nothing
)

qcp_kwargs = Dict(
    :linear_solver => "mumps",
    :ipopt_options => ops,
    :jacobian_structure_file => nothing,
    :save_jacobian_structure => false,
    :hessian_approximation => true,
    :eval_hessian => false,
    :evaluator_verbose => false,
)
;

# Bang-bang control
------

Try to implement an X gate when you have a slow drift.  A simple X rotation won't cut it. 

In [ ]:
H_X = GATES[:X]
H_Y = GATES[:Y]
H_Z = GATES[:Z]

# X gate
U_goal = [1 1; 1 -1] / √2

# control_system = QuantumSystem(0.25 * H_Z, [H_X, H_Y])
control_system = QuantumSystem([H_X, H_Y])

## Standard Solution

In [ ]:
T = 50
Δt = 0.2

prob = UnitarySmoothPulseProblem(
    control_system, U_goal, T, Δt,
    ipopt_options=ops, hessian_approximation=true, free_time=false
)

In [ ]:
solve!(prob; max_iter=200)
println("\nInfidelity: ", 1 - unitary_fidelity(prob))

In [ ]:
f = Figure(resolution = (800, 400))
colors = (:blue, :red)
for (i, row) in enumerate(eachrow(prob.trajectory[:a]))
    ax = f[1, i] = Axis(f, xlabel = "Time", ylabel = "Control amplitude $(i == 1 ? "X" : "Y")")
    lines!(ax, Δt .* (1:T), row, color=colors[i], linewidth=3)
    band!(ax, Δt .* (1:T), zeros(size(row)), row, color = (colors[i], 0.1))
end
f

## Modified Solution

We don't have to control on acceleration. Here is how we build a NamedTrajectory under the hood.

In [ ]:
# NamedTrajectory
# ===============
components = (
    Ũ⃗ = prob.trajectory[:Ũ⃗],
    a = prob.trajectory[:a],
    da = prob.trajectory[:da],
)

bounds = (
    a = ([-0.5, -0.5], [0.5, 0.5]),
    da = ([-0.5, -0.5], [0.5, 0.5]),
)

# Tell the NamedTrajectory what to use as state and what to use as control
traj = NamedTrajectory(
    components;
    controls=(:da,),
    timestep=Δt,
    bounds=bounds,
    initial=prob.trajectory.initial,
    final=prob.trajectory.final,
    goal=prob.trajectory.goal
)

# Cost
# ====
Q = 100.0
R = 1e-4

J = UnitaryInfidelityObjective(:Ũ⃗, traj, Q)
J += QuadraticRegularizer(:a, traj, R)
J += QuadraticRegularizer(:da, traj, R)
obj, constraint = L1Regularizer(:da, traj, R=[1., 1.])
J += obj


# Dynamics constraints
# ====================
integrators = [
    UnitaryPadeIntegrator(:Ũ⃗, :a, traj, control_system; integrator_kwargs...),
    DerivativeIntegrator(:a, :da, traj),
]

In [ ]:
velocity_prob = QuantumControlProblem(
    control_system, traj, J, integrators;
    constraints=AbstractConstraint[constraint],
    qcp_kwargs...
)

In [ ]:
solve!(velocity_prob; max_iter=200)
println("\nInfidelity: ", 1 - unitary_fidelity(velocity_prob))

In [ ]:
f = Figure(resolution = (800, 400))
colors = (:blue, :red)
ax = f[1, 1] = Axis(f, xlabel = "Time", ylabel = "Control amplitude", title="Control")

for (i, row) in enumerate(eachrow(velocity_prob.trajectory[:a]))
    lines!(ax, Δt .* (1:T), row, color=colors[i], linewidth=3, label="Control $(i == 1 ? "X" : "Y")")
    band!(ax, Δt .* (1:T), zeros(size(row)), row, color=(colors[i], 0.1))
end
Legend(f[1,2], ax)
f

In [ ]:
f = Figure(resolution = (900, 400))
colors = (:blue, :red)

for (col, problem) in enumerate([prob, velocity_prob])
    ax = f[1, col] = Axis(f, xlabel = "Time", ylabel = "Control amplitude", title="Velocity $(col == 1 ? "Smooth" : "L1")")
    for (i, row) in enumerate(eachrow(problem.trajectory[:da]))
        lines!(ax, Δt .* (1:T), row, color=colors[i], linewidth=3, label="Control $(i == 1 ? "X" : "Y")")
        band!(ax, Δt .* (1:T), zeros(size(row)), row, color=(colors[i], 0.1))
    end
    Legend(f[1, 3], ax)
end
f